## Previously defined functions

In [1]:
## Here is the likelihood function for a Logit
calcllik<-function(b,data){     
    
  y<-data[,1]
  x<-data[,2]

  pi<- 1/(1+exp(-b[1] - b[2]*x))        # Here is the systematic component
  llik<-y * log(pi) + (1-y) * log(1-pi) # Here is the stocastic component
    
  return(-llik)
}

## Differentially private mean release
gaussianReleaseNoise <- function(size=1, sensitivity, epsilon, delta){
	scale <- sensitivity *sqrt(2*log(1.25/delta))/ epsilon
	noise <- rnorm(n=size, mean=0, sd=scale)
	return(noise)
}

## Bound/Censor/Clip a variable to a range
clip <- function(x, lower, upper){
	x.clipped <- x
	x.clipped[x.clipped<lower] <- lower
	x.clipped[x.clipped>upper] <- upper
	return(x.clipped)	
}

## Run with actual data

In [2]:
library("foreign")
library("pracma")
PUMSdata <- read.csv(file="https://raw.githubusercontent.com/privacytoolsproject/cs208/master/data/MaPUMS5full.csv")     


mydata<-PUMSdata[c("married","educ")]

output <- glm(married ~ educ, family="binomial", data=mydata)

print(summary(output))

write.csv(coef(output), file="non_private_results.csv")

Warning message:
“package ‘pracma’ was built under R version 3.5.2”


Call:
glm(formula = married ~ educ, family = "binomial", data = mydata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.4551  -1.2135   0.9831   1.0769   1.4144  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.619976   0.015070  -41.14   <2e-16 ***
educ         0.078282   0.001404   55.76   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 333086  on 241829  degrees of freedom
Residual deviance: 329918  on 241828  degrees of freedom
AIC: 329922

Number of Fisher Scoring iterations: 4



## Stocastic Gradient Descent

In [3]:
# Calculate the gradient at a point in the parameter space
calcgradient <- function(B, C, theta, fun){
	dx <- 0.0001

	out1 <-	eval(fun(b=theta, data=B))
	out2 <- eval(fun(b=theta + c(0,dx), data=B))
	out3 <- eval(fun(b=theta + c(dx,0), data=B))

	Del.1 <- (out3-out1)/dx                   
	Del.1 <- clip(Del.1, lower=(-C), upper=C) 
	mean.Del.1 <- mean(Del.1)

	Del.2 <- (out2-out1)/dx                   
	Del.2 <- clip(Del.2, lower=(-C), upper=C) 
	mean.Del.2 <- mean(Del.2)

	return(c(mean.Del.1,mean.Del.2))
}

In [5]:
epsilon_seq <- logspace(-2, 1, 30)
num_trials <- 10
results <- matrix(NA, nrow=length(epsilon_seq)*num_trials, ncol=3)

for (e in 1:length(epsilon_seq)) {
    for (f in 1:num_trials) {
        epsilon <- epsilon_seq[e]
        N <- nrow(mydata)
        L <- round(sqrt(nrow(mydata)))     # This is the recommended batch size
        steps <- L  

        ## Shuffle the data
        index <- sample(1:nrow(mydata))
        mydata <- mydata[index,]

        theta <- c(0,0)   # Starting parameters
        C <- 10			  # Interval to clip over
        nu <- c(1,0.01)   # Learning speeds

        history <- matrix(NA, nrow=steps+1, ncol=2)
        history[1,] <- theta

        # Iterate one step of SGD
        for (i in 1:steps) {
            startB <- ((i-1)*L+1)
            if(i<L){
                stopB <- i*L
            }else{
                stopB <- nrow(mydata)
            }

            index<-sample(1:nrow(mydata),L)
            B <- mydata[startB:stopB, ]
            
            #local DP gradient
            del_local <- 0
            for (j in 1:nrow(B)){
                non_private_gradient <- calcgradient(B[j,], C, theta, fun=calcllik)
                noisy_gradient <- non_private_gradient + gaussianReleaseNoise(size=2, sensitivity=2*L*C/N, epsilon=epsilon/2, delta=1e-6)
                del_local <- del_local + noisy_gradient
            }
            del_local <- del_local / nrow(B)
            
            theta <- theta - del_local*nu
            history[i+1,] <- theta
        }
        
        results[10*(e-1)+f,] <- c(epsilon, history[i+1,])
        message(results[10*(e-1)+f,])
    }

}
                  
print(results)
write.csv(results, file = "local_results.csv")

0.01-2.572823957134160.148149739601315
0.01-6.819166970016790.672925809681945
0.01-1.457243940897980.180902095361191
0.01-30.86919180320582.70912930003679
0.013.010732461877950.305855627451614
0.01-11.83115184652231.09340231116534
0.0129.7514999574723-2.33531795682445
0.0116.1740209842666-1.63462031436959
0.0116.8655403766554-1.26075781804032
0.01-5.561507242577850.683055803335788
0.01268961003167920.9749056160015220.0639710208038931
0.01268961003167926.00144082101032-0.518298074609458
0.0126896100316792-36.52106601575863.32570630913341
0.012689610031679217.3247459920812-1.14790827462226
0.01268961003167925.82355035193623-0.565172200316933
0.012689610031679213.6483262118221-0.839419867456779
0.012689610031679212.6601089240689-0.90981459143599
0.01268961003167920.1059805533672320.0265716527596328
0.01268961003167926.03364234711174-0.678046932335487
0.0126896100316792-6.94436880850220.753167219507727
0.0161026202756094-6.866027561355440.663240183645185
0.0161026202756094-1.14653543695664

              [,1]         [,2]          [,3]
  [1,]  0.01000000  -2.57282396  0.1481497396
  [2,]  0.01000000  -6.81916697  0.6729258097
  [3,]  0.01000000  -1.45724394  0.1809020954
  [4,]  0.01000000 -30.86919180  2.7091293000
  [5,]  0.01000000   3.01073246  0.3058556275
  [6,]  0.01000000 -11.83115185  1.0934023112
  [7,]  0.01000000  29.75149996 -2.3353179568
  [8,]  0.01000000  16.17402098 -1.6346203144
  [9,]  0.01000000  16.86554038 -1.2607578180
 [10,]  0.01000000  -5.56150724  0.6830558033
 [11,]  0.01268961   0.97490562  0.0639710208
 [12,]  0.01268961   6.00144082 -0.5182980746
 [13,]  0.01268961 -36.52106602  3.3257063091
 [14,]  0.01268961  17.32474599 -1.1479082746
 [15,]  0.01268961   5.82355035 -0.5651722003
 [16,]  0.01268961  13.64832621 -0.8394198675
 [17,]  0.01268961  12.66010892 -0.9098145914
 [18,]  0.01268961   0.10598055  0.0265716528
 [19,]  0.01268961   6.03364235 -0.6780469323
 [20,]  0.01268961  -6.94436881  0.7531672195
 [21,]  0.01610262  -6.86602756  0